# Process SmolLM Corpus

In [ ]:
from datasets import load_dataset
import boto3
import gzip,os

# cv2 = load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2")
# fed = load_dataset("HuggingFaceTB/smollm-corpus", "fineweb-edu-dedup")
# pe = load_dataset("HuggingFaceTB/smollm-corpus", "python-edu")

# os.environ["AWS_SECRET_ACCESS_KEY"]="YOURKEY"
# os.environ["AWS_ACCESS_KEY_ID"]="YOURKEY"

In [ ]:
fed4 = fed.filter(lambda x: x['metadata']['int_score'] >= 4, num_proc=120)
# fed4=load_dataset("chengjunyan1/smollm-10", "fineweb-edu-dedup")

In [ ]:
pe4 = pe.filter(lambda x: x['score'] > 4.16, num_proc=120)
pe4['train'].num_rows,pe4['train'].num_rows/pe['train'].num_rows

In [ ]:
from datasets import dataset_dict

cv2_ratio=(fed4['train'].num_rows/fed['train'].num_rows+pe4['train'].num_rows/pe['train'].num_rows)/2
cv2_samples=int(cv2.num_rows['train']*cv2_ratio)
cv2_10=cv2['train'].shuffle(seed=42).select(range(cv2_samples))
cv2_10=dataset_dict.DatasetDict({'train':cv2_10})

In [ ]:
len_all=fed4['train'].num_rows+pe4['train'].num_rows+cv2_10['train'].num_rows
len_all_full=fed['train'].num_rows+pe['train'].num_rows+cv2['train'].num_rows
print(fed4['train'].num_rows/len_all, pe4['train'].num_rows/len_all, cv2_10['train'].num_rows/len_all)
print(fed['train'].num_rows/len_all_full, pe['train'].num_rows/len_all_full, cv2['train'].num_rows/len_all_full)

In [ ]:
from huggingface_hub import login

login()

In [ ]:
# pe4.push_to_hub("chengjunyan1/smollm-10","python-edu")
# fed4.push_to_hub("chengjunyan1/smollm-10","fineweb-edu-dedup")
# cv2_10.push_to_hub("chengjunyan1/smollm-10","cosmopedia-v2")

# Verify processed corpus

In [ ]:
from datasets import load_dataset,DatasetDict
import boto3
import gzip,os
import functools as ft


# os.environ["AWS_SECRET_ACCESS_KEY"]="YOURKEY"
# os.environ["AWS_ACCESS_KEY_ID"]="YOURKEY"

In [ ]:
fed10=load_dataset("chengjunyan1/smollm-10","fineweb-edu-dedup")
cv10=load_dataset("chengjunyan1/smollm-10","cosmopedia-v2")

In [ ]:
DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download

session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
s3 = session.client("s3")
bucket_name = "softwareheritage"

def download_contents(blob_id):
    key = f"content/{blob_id}"
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    with gzip.GzipFile(fileobj=obj['Body']) as fin:
        content = fin.read().decode("utf-8", errors="ignore")
    return {"text": content}

def download_python_edu():
    ds = load_dataset("chengjunyan1/smollm-10", "python-edu", split="train", num_proc=DEFAULT_NUM_PROC)
    ds = ds.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
    ds = DatasetDict({"train": ds})
    return ds

pe10=download_python_edu()

In [ ]:
from datasets import concatenate_datasets,DatasetDict
import numpy as np


def resample_dataset(dataset, weight):
    num_samples = int(len(dataset) * weight)
    indices = np.random.choice(len(dataset), num_samples, replace=True)
    return dataset.select(indices)

def combine_datasets(dataset_dicts, weights:dict=None): # weights e.g. {'train':[1.5,1.0]}
    combined_dict = {}
    
    # Initialize weights if not provided
    for dataset_dict in dataset_dicts:
        for key, dataset in dataset_dict.items():
            if key in combined_dict:
                combined_dict[key] = concatenate_datasets([combined_dict[key], dataset])
            else:
                combined_dict[key] = dataset

    # Apply weights by resampling the datasets
    for key in combined_dict:
        datasets = []
        for idx,dataset in enumerate(dataset_dicts):
            if key in dataset:
                if weights is None or key not in weights or weights[key][idx] == 1.0:
                    datasets.append(dataset[key])
                else:
                    resampled_dataset = resample_dataset(dataset[key], weights[key][idx])
                    datasets.append(resampled_dataset)
        combined_dict[key] = concatenate_datasets(datasets)
    
    return DatasetDict(combined_dict)


dataset_dicts=[pe10,fed10,cv10]
combined=combine_datasets(dataset_dicts)

# Create test split based on the processed corpus

In [3]:
import os
from datasets import load_dataset,DatasetDict
import boto3
import gzip,os
from huggingface_hub import login


login('YOURKEY')

DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download

os.environ["AWS_ACCESS_KEY_ID"]="YOURKEY"
os.environ["AWS_SECRET_ACCESS_KEY"]="YOURKEY"

session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
s3 = session.client("s3")
bucket_name = "softwareheritage"

def download_contents(blob_id):
    key = f"content/{blob_id}"
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    with gzip.GzipFile(fileobj=obj['Body']) as fin:
        content = fin.read().decode("utf-8", errors="ignore")
    return {"text": content}

def download_python_edu():
    ds = load_dataset("chengjunyan1/smollm-10", "python-edu", split="train", num_proc=DEFAULT_NUM_PROC)
    ds = ds.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
    ds = DatasetDict({"train": ds})
    return ds

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/junyanc/.cache/huggingface/token
Login successful


In [4]:
pe=load_dataset("HuggingFaceTB/smollm-corpus", "python-edu", num_proc=DEFAULT_NUM_PROC//8)
fed=load_dataset("HuggingFaceTB/smollm-corpus", "fineweb-edu-dedup", num_proc=DEFAULT_NUM_PROC//8)
cv=load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2", num_proc=DEFAULT_NUM_PROC//8)
fed10=load_dataset("chengjunyan1/smollm-10","fineweb-edu-dedup",num_proc=DEFAULT_NUM_PROC//8)
cv10=load_dataset("chengjunyan1/smollm-10","cosmopedia-v2",num_proc=DEFAULT_NUM_PROC//8)
pe10=download_python_edu()

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/234 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/1934 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

...

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/50 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Parameter 'function'=<function download_contents at 0x7f7687a44ae0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map (num_proc=512):   0%|          | 0/864074 [00:00<?, ? examples/s]

In [5]:
# random sample 2 GB of data from each dataset
smollm_size=673
test_ratio=1/smollm_size
pe_test_lines=int(pe['train'].num_rows*test_ratio)
fed_test_lines=int(fed['train'].num_rows*test_ratio)
cv_test_lines=int(cv['train'].num_rows*test_ratio)
pe_shuffle=pe['train'].shuffle(seed=42)
fed_shuffle=fed['train'].shuffle(seed=42)
cv_shuffle=cv['train'].shuffle(seed=42)
pe_test=pe_shuffle.select(range(pe_test_lines))
pe_eval=pe_shuffle.select(range(pe_test_lines,pe_test_lines*2))
pe_test=pe_test.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
pe_eval=pe_eval.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
fed_test=fed_shuffle.select(range(fed_test_lines))
fed_eval=fed_shuffle.select(range(fed_test_lines,fed_test_lines*2))
cv_test=cv_shuffle.select(range(cv_test_lines))
cv_eval=cv_shuffle.select(range(cv_test_lines,cv_test_lines*2))

print('Python-Edu:',pe10.num_rows,pe_eval.num_rows,pe_test.num_rows)
print('FineWeb-Edu-Dedup:',fed10.num_rows,fed_eval.num_rows,fed_test.num_rows)
print('Cosmopedia-V2:',cv10.num_rows,cv_eval.num_rows,cv_test.num_rows)

Map (num_proc=512):   0%|          | 0/11409 [00:00<?, ? examples/s]

Map (num_proc=512):   0%|          | 0/11409 [00:00<?, ? examples/s]

Python-Edu: {'train': 864074} 11409 11409
FineWeb-Edu-Dedup: {'train': 22701367} 282567 282567
Cosmopedia-V2: {'train': 4537737} 58148 58148


In [6]:
pe_test,pe_eval,fed_test,fed_eval,cv_test,cv_eval

(Dataset({
     features: ['blob_id', 'repo_name', 'path', 'length_bytes', 'score', 'int_score', 'text'],
     num_rows: 11409
 }),
 Dataset({
     features: ['blob_id', 'repo_name', 'path', 'length_bytes', 'score', 'int_score', 'text'],
     num_rows: 11409
 }),
 Dataset({
     features: ['text', 'id', 'metadata'],
     num_rows: 282567
 }),
 Dataset({
     features: ['text', 'id', 'metadata'],
     num_rows: 282567
 }),
 Dataset({
     features: ['prompt', 'text', 'token_length', 'audience', 'format', 'seed_data'],
     num_rows: 58148
 }),
 Dataset({
     features: ['prompt', 'text', 'token_length', 'audience', 'format', 'seed_data'],
     num_rows: 58148
 }))

In [8]:
# remove verbatims
import multiprocessing as mp
def get_text(x):
    return x['text']
with mp.Pool(mp.cpu_count()) as pool:
    pe_text=pool.map(get_text, pe_test)+pool.map(get_text, pe_eval)
    fed_text=pool.map(get_text, fed_test)+pool.map(get_text, fed_eval)
    cv_text=pool.map(get_text, cv_test)+pool.map(get_text, cv_eval)

In [9]:
pe_train=pe10.filter(lambda x: x['text'] not in pe_text, num_proc=DEFAULT_NUM_PROC//4-8)
cv_train=cv10.filter(lambda x: x['text'] not in cv_text, num_proc=DEFAULT_NUM_PROC//4-8)
fed_train=fed10.filter(lambda x: x['text'] not in fed_text, num_proc=DEFAULT_NUM_PROC//4-8)

Filter (num_proc=120):   0%|          | 0/864074 [00:00<?, ? examples/s]

Filter (num_proc=120):   0%|          | 0/4537737 [00:00<?, ? examples/s]

Filter (num_proc=120):   0%|          | 0/22701367 [00:00<?, ? examples/s]

In [ ]:
pe_train,pe10,fed_train,fed10,cv_train,cv10

NameError: name 'pe_train' is not defined

In [ ]:
pe10_corpus=DatasetDict({'train':pe10['train'],'eval':pe_eval,'test':pe_test})
fed10_corpus=DatasetDict({'train':fed10['train'],'eval':fed_eval,'test':fed_test})
cv10_corpus=DatasetDict({'train':cv10['train'],'eval':cv_eval,'test':cv_test})

In [ ]:
pe10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","python-edu")
fed10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","fineweb-edu-dedup")
cv10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","cosmopedia-v2")

# Make train and test splits from scratch

In [1]:
import os
from datasets import load_dataset,DatasetDict
import boto3
import gzip,os
from huggingface_hub import login
DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download

os.environ["AWS_ACCESS_KEY_ID"]="YOURKEY"
os.environ["AWS_SECRET_ACCESS_KEY"]="YOURKEY"

session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
s3 = session.client("s3")
bucket_name = "softwareheritage"

def download_contents(blob_id):
    key = f"content/{blob_id}"
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    with gzip.GzipFile(fileobj=obj['Body']) as fin:
        content = fin.read().decode("utf-8", errors="ignore")
    return {"text": content}

def download_python_edu():
    ds = load_dataset("chengjunyan1/smollm-10", "python-edu", split="train", num_proc=DEFAULT_NUM_PROC)
    ds = ds.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
    ds = DatasetDict({"train": ds})
    return ds

In [2]:
pe=load_dataset("HuggingFaceTB/smollm-corpus", "python-edu", num_proc=DEFAULT_NUM_PROC//8)
fed=load_dataset("HuggingFaceTB/smollm-corpus", "fineweb-edu-dedup", num_proc=DEFAULT_NUM_PROC//8)
cv=load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2", num_proc=DEFAULT_NUM_PROC//8)

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/234 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/1934 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/480 [00:00<?, ?it/s]

In [3]:
# random sample 2 GB of data from each dataset
smollm_size=673
test_ratio=1/smollm_size
pe_test_lines=int(pe['train'].num_rows*test_ratio)
fed_test_lines=int(fed['train'].num_rows*test_ratio)
cv_test_lines=int(cv['train'].num_rows*test_ratio)
pe_shuffle=pe['train'].shuffle(seed=42)
fed_shuffle=fed['train'].shuffle(seed=42)
cv_shuffle=cv['train'].shuffle(seed=42)
pe_test=pe_shuffle.select(range(pe_test_lines))
pe_eval=pe_shuffle.select(range(pe_test_lines,pe_test_lines*2))
pe_train=pe_shuffle.select(range(pe_test_lines*2,pe_shuffle.num_rows))
fed_test=fed_shuffle.select(range(fed_test_lines))
fed_eval=fed_shuffle.select(range(fed_test_lines,fed_test_lines*2))
fed_train=fed_shuffle.select(range(fed_test_lines*2,fed_shuffle.num_rows))
cv_test=cv_shuffle.select(range(cv_test_lines))
cv_eval=cv_shuffle.select(range(cv_test_lines,cv_test_lines*2))
cv_train=cv_shuffle.select(range(cv_test_lines*2,cv_shuffle.num_rows))

print('Python-Edu:',pe_train.num_rows,pe_eval.num_rows,pe_test.num_rows)
print('FineWeb-Edu-Dedup:',fed_train.num_rows,fed_eval.num_rows,fed_test.num_rows)
print('Cosmopedia-V2:',cv_train.num_rows,cv_eval.num_rows,cv_test.num_rows)

Python-Edu: 7655630 11409 11409
FineWeb-Edu-Dedup: 189602871 282567 282567
Cosmopedia-V2: 39017704 58148 58148


In [ ]:
pe10 = pe_train.filter(lambda x: x['score'] > 4.16, num_proc=DEFAULT_NUM_PROC//4-8)
fed10 = fed_train.filter(lambda x: x['metadata']['int_score'] >= 4, num_proc=DEFAULT_NUM_PROC//4-8)
cv2_ratio=(fed10['train'].num_rows/fed_train['train'].num_rows+pe10['train'].num_rows/pe_train['train'].num_rows)/2
cv2_samples=int(cv_train.num_rows['train']*cv2_ratio)
cv10=cv_train['train'].shuffle(seed=42).select(range(cv2_samples))
cv10=dataset_dict.DatasetDict({'train':cv10})

In [ ]:
pe10_corpus=DatasetDict({'train':pe10['train'],'eval':pe_eval,'test':pe_test})
fed10_corpus=DatasetDict({'train':fed10['train'],'eval':fed_eval,'test':fed_test})
cv10_corpus=DatasetDict({'train':cv10['train'],'eval':cv_eval,'test':cv_test})

In [ ]:
pe10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","python-edu")
pe10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","fineweb-edu-dedup")
pe10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","cosmopedia-v2")